In [ ]:
import numpy as np
import scipy as sp
from iminuit import Minuit
import pandas as pd
idx = pd.IndexSlice

import clapy as cla
import clapy.plot
import clapy.utils

import matplotlib.pyplot as plt
%matplotlib inline


import seaborn as sb
import cython

In [ ]:
import cython
from iminuit import Minuit
%load_ext Cython

In [ ]:
%%cython --force
# distutils: language = c++

#use --annotate if you wonder what kind of code it generates 
cimport cython
import numpy as np
cimport numpy as np #overwritten those from python with cython
from libc.math cimport exp, M_PI, sqrt, log
from iminuit.util import describe, make_func_code
from libcpp.map cimport map     
import pybrdu as brdu
import scipy as sp



@cython.embedsignature(True)#dump the signatre so describe works
cpdef fitfunc(double t,double Tc,double r,double GF):
    cdef double res = 0
    if t<(Tc-Tc*r):
        res = GF/Tc*(t+Tc*r)
    else:
        res =  GF
    return res 
'''
@cython.embedsignature(True)#dump the signatre so describe works
cpdef double mypdf(double x, double mu, double std):
    #cpdef means generate both c function and python function
    cdef double norm = 1./(sqrt(2*M_PI*std))
    cdef double ret = exp(-1*(x-mu)*(x-mu)/(2.*std))*norm
    return ret
'''
@cython.embedsignature(True)#dump the signatre so describe works
cpdef double mypdfln(double x, double mu, double std):
    #cpdef means generate both c function and python function
    cdef double norm = (sqrt(2*M_PI*std*std))
    cdef double ret = (-1*(x-mu)*(x-mu)/(2.*std*std))-log(norm)
    return ret




cdef class QuickAndDirtyLogLHB:#cdef is here to reduce name lookup for __call__
    cdef np.ndarray data
    cdef np.ndarray err
    cdef np.ndarray t
    cdef int ndata
    
    def __init__(self,data,t):
        self.data = data
        self.t = t
        self.ndata = len(data)
    
    @cython.embedsignature(True)#you need this to dump function signature in docstring
    def compute(self, double Tc,double r,double GF,double s):
        #this line is a cast not a copy. Let cython knows mydata will spit out double
        cdef np.ndarray[np.double_t, ndim=1] mydata = self.data
        cdef np.ndarray[np.double_t, ndim=1] myt = self.t
        cdef double loglh = 0.
        cdef double lmu = 0.
        cdef double ler = 0.
        for i in range(self.ndata):
            lmu = fitfunc(myt[i],Tc,r,GF)
            loglh -= mypdfln(mydata[i],lmu,s)
        return loglh



In [ ]:
t = 0.4
ss = 0.2
sc = 0.3
gf = 0.95
tc = 1.0
rs = 0.3
d = cla.utils.dist()
x = np.arange(0,100+1)
pdfs = list()
pdfm = list()
data = list()
#ts = [0.05,0.25,0.5,0.75,1.1]
#ts = np.linspace(0.2,2.0,100)
#ts = np.linspace(0.2,2.0,4*30+1)
#ts = np.linspace(0.2-0.015*10,2.0,4*30+1+10)
ts = np.linspace(0.01,2.0,797)


default_args = {
        'samples': 10000,
        'nCells': x[-1],
        'mCells': x[-1],
        'GF': gf,
        'G1': tc*((1-rs)*0.5),
        'S': tc*(rs),
        'G2M': tc*((1-rs)*0.5),
        'sSamples' : sc,
        'sPopulation' : ss 
}


#times=np.linspace(0.01,1.2,9)
tim = np.ravel(np.array(ts)[:,np.newaxis]*np.ones(default_args['samples']))


sizeB = 3.25*1.1
sizeH = 2.5*1.1
fonts = 0.9
color_pr = np.array([0.5647058823529412, 0.9333333333333333, 0.5647058823529412]) - np.array([0.4,0.1,0.4])
color_no = '#CC79A7'

from matplotlib import rc
rc('text', usetex=True)
font ='DejaVu Serif'

In [ ]:
#gen data (Takes some time)
'''
for t in ts:
    #simulations
    data.append(np.array(cla.run(seed=int(np.random.rand()*1000),mode=1,times=[t],**default_args)) )
    #pdfs
    pdfs.append( [d.pmf_f(x[-1],tc,rs,gf,sc,ss,t,i) for i in x] )
    #means
    pdfm.append( d.pmf_mean(x[-1],tc,rs,gf,sc,ss,t) )
    
datsa = np.array([ 100*np.array(data).ravel() , (ts*np.ones((default_args['samples'],len(ts)))).T.ravel()])


dy = ts[1]-ts[0]
mini = np.min(datsa,axis=1)
maxi = np.max(datsa,axis=1)
lenx =  101
leny =  len(ts)
grid = np.zeros((leny,lenx))
print(lenx,leny,mini,maxi)
for i in datsa.T:
    grid[ int(np.round((i[1]-mini[1])/dy)),int(np.round(i[0])) ] += 1
    
np.savez_compressed('paper_img_data',data=data,pdfs=pdfs,pdfm=pdfm,grid=grid)
'''

In [ ]:
npz  = np.load('paper_img_data.npz')
data_t = npz['data']
pdfs = npz['pdfs']
pdfm = npz['pdfm']
grid = npz['grid']

In [ ]:
#some plot settings
s_s = int(np.round(149/4.0))
ts = np.linspace(0.01,2.0,797)
s_t = 149
se_t = 149*5
sa_t = 0


In [ ]:
sb.set(context="paper",style='ticks',font=font,font_scale=fonts)


fig = plt.figure(2,figsize=(sizeB, sizeH))
fig.clear()
fig.subplots_adjust(left=0.18,bottom=0.2,top=0.95)

ttt = np.linspace(ts[0],ts[-1],1000)
ax = fig.gca()

ax.set_ylim((0,100))
ax.set_xlim((0,2))
ax.set_xlabel('time')
ax.set_ylabel('\#labeled cells fit $k$')



tsd = ts
ysd = np.array([100*np.mean(i) for i in data_t])


error =[]
for i in data_t:
    xxx,yyy = cla.utils.ecdf(100*i)
    p95a = int(np.round(default_args['samples']*0.025))
    p95e = int(np.round(default_args['samples']*0.975))
    error.append([xxx[p95a],xxx[p95e]])
error = np.abs(np.array(error).T - ysd)


lh = QuickAndDirtyLogLHB(np.array([np.mean(i) for i in data_t])[sa_t:se_t:s_t],ts[sa_t:se_t:s_t])
mi_old = Minuit(lh.compute, Tc=1.0, r=0.3,GF=1,s=0.1,\
                      error_r=0.1,error_GF=0.1,error_Tc=0.1,error_s=0.1,\
           limit_Tc=(0.00001,2), limit_r=(0.00001,1),limit_GF=(0,1),limit_s=(0.0001,1),\
           errordef=0.5,print_level=2)
mi_old.migrad()
res = mi_old.values
args = {
        'GF': res['GF'],
        'G1': res['Tc']*((1-res['r'])*0.5),
        'S': res['Tc']*res['r'],
        'G2M': res['Tc']*((1-res['r'])*0.5)
}




plt.errorbar(tsd[::s_s],ysd[::s_s],error[:,::s_s],fmt='.',color='black',label="nummerical sol.",zorder=2 )

plt.plot(ttt,100*cla.utils.brdu_model(ttt,**default_args),label="Nowakowski",color=color_no,zorder=5)

plt.plot(ttt,100*cla.utils.brdu_model(ttt,**args),':',color=color_no,label="fit",zorder=6)

plt.plot(ts[sa_t:se_t:s_t],[100*np.mean(i) for i in data_t][sa_t:se_t:s_t],'^',color='black',label="fit points",zorder=7 )

plt.plot(ts,100*np.array(pdfm),label='analytic sol.',color=color_pr,zorder=4)




handles, labels = ax.get_legend_handles_labels()
plt.legend(reversed(handles), reversed(labels), loc='lower right')



In [ ]:
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig = plt.figure(3,figsize=(sizeB, sizeH*1.28))
fig.clear()

sb.set(context="paper",style='ticks',font=font,font_scale=fonts)

ax1 = plt.subplot2grid((3,1),(0,0),rowspan=2)
ax2 = plt.subplot2grid((3,1),(2,0))
ax1.plot(-3,-3,"k-",label="Solution")
ax1.plot(-3,-3,"k.",label="Simulation")

s = 149
a = 0
b = 149*3

x = np.arange(0,100+1)
def cmap(x):
    return sb.color_palette(palette='colorblind',n_colors=3)[int((x*1000)%3)]

norm = mpl.colors.Normalize(vmin=ts[a:b:s][0], vmax=ts[a:b:s][-1])

ss = 1

for i_n,(i,d,p) in enumerate(zip(ts[a:b:s],data_t[a:b:s],pdfs[a:b:s])):
    h = ax1.hist(d[::ss]*x[-1],density=True,bins=np.arange(x[0]-0.5,x[-1]+0.6),color=cmap(norm(i)),alpha=0.5,lw=0 )
    ax1.plot(x,p,color=cmap(norm(i)),drawstyle='steps-mid')
    ax1.bar(x,p,1,linewidth=0.5,color=(0,0,0,0),edgecolor=cmap(norm(i)))
    if i_n==2:
        ax1.annotate("t={:1.2f}".format(i),xy=(np.mean(d*x[-1]),h[0].max()) ,xytext=(np.mean(d*x[-1])-17,h[0].max()-0.005 ),\
                color=cmap(norm(i)))
    else:
        ax1.annotate("t={:1.2f}".format(i),xy=(np.mean(d*x[-1]),h[0].max()) ,xytext=(np.mean(d*x[-1])-10,h[0].max()+0.005 ),\
                color=cmap(norm(i)))
#ax1.legend()
ax1.set_xlim(0,100)
ax1.set_ylim(0,0.11)
  
    
ax2.plot(-3,-3,"k-",label="Solution")
ax2.plot(-3,-3,"k.",label="Simulation")


for i,d,p in zip(ts[a:b:s],data_t[a:b:s],pdfs[a:b:s]):

    ax2.plot(x,np.cumsum(p),"-",color=cmap(norm(i)),drawstyle='steps-mid',lw=1 )
    xx,y = cla.utils.ecdf(d[::ss]*x[-1])
    ux,cx = np.unique(xx,return_counts=True)
    ax2.plot(ux,[y[i:j].max() for i,j in zip(np.cumsum(cx)-cx[0],np.cumsum(cx))],'.',color=cmap(norm(i)) )
    #ax2.annotate("t={}".format(i),xy=(np.mean(xx),np.mean(y)),xytext=(np.mean(xx)-10,(2-i)*0.3+0.2 ),\
    #            color=cmap(norm(i)),bbox={'facecolor':'white', 'alpha':1, 'pad':2})

#ax2.legend()
ax2.set_xlim(0,100)
ax2.set_ylim(-0.05,1.15)



ax1.set_xticks([])
ax1.set_xlabel('')

ax2.set_xlabel('\#labeled cells $k$')

ax1.set_ylabel('$P(K = k)$')
ax2.set_ylabel('$P(K\leq k)$')

divider = make_axes_locatable(ax1)
lax = divider.append_axes("top", size="5%", pad=0.035)
lax.set_xticks([])
lax.set_yticks([])
lax.set_frame_on(False)
lax.set_xlim(0,100)
a = 20
a3 = 37
s = 1.2
a2 = 2.5*s
at= 6

lax.text(a+at,0,'analytic')
lax.bar(a+s*np.array([0,1]),[2.8,5],s*1,linewidth=0.5,color=(0,0,0,0),edgecolor='black')
lax.plot(a+s*np.array([-0.50,-0.5,0.5,0.5,1.5,1.5]),[0,2.8,2.8,5,5,0],color='black')
lax.plot(a+a2+s*np.array([-0.,1.]),[2.5,2.5],lw=1,color='k')

lax.text(a+a3+at,0,'numerical')
lax.bar(a+a3+s*np.array([0,1]),[2.8,5],s*1,linewidth=0.0,color=(0,0,0,0),edgecolor=None,alpha=0.5)
lax.plot(a+a3+a2+s*np.array([0.5]),[2.5],'k.')



plt.tight_layout()
fig.subplots_adjust(hspace=0)

In [ ]:
import matplotlib.transforms as plttransforms
import matplotlib.lines as pltlines

ho_abc = 0.35
ho_abc2 = 0.45
alpha_no = 0.4
alpha_pr = 0.3
#load data
with pd.HDFStore("data_minuit_1000") as store:
    print(store.keys())
    data_s = store['minuit_full_t001_15_s5_GF095_m5']
    data_sv = store['minuit_full_t001_15_s100_GF095_m100']
    

#set dataframe
f = plt.figure(4,figsize=(sizeB*2, sizeH*2.5))
f.clear()

sb.set(context="paper",style='ticks',font=font,font_scale=fonts)
fh = f.get_figheight() 
fb = f.get_figwidth() 

f.subplots_adjust(left=0.06,right=0.99,hspace=0.28,top=0.94)
f.clear()
p = {'Tc' : 1,'r' : 0.3 , 'ss':0.2,'sc' :0.3,'GFf':0.95,'m' : 5 }

A = 'full' 
B = 'old' 
#plot swarm wit N_data points
N_data = 100
#plot and calc
data,sdata = cla.utils.calc_statistic(data_sv,p)
axs = (3,10)

ax_1 = plt.subplot2grid(axs,(0,0),colspan=2)
ax_2 = plt.subplot2grid(axs,(0,2),colspan=2)
ax_3 = plt.subplot2grid(axs,(0,4),colspan=2)
ax_4 = plt.subplot2grid(axs,(0,6),colspan=2)
ax_5 = plt.subplot2grid(axs,(0,8),colspan=2)

ax_6 = plt.subplot2grid(axs,(1,0),colspan=2)
ax_7 = plt.subplot2grid(axs,(1,2),colspan=2)
ax_8 = plt.subplot2grid(axs,(1,4),colspan=2)
ax_9 = plt.subplot2grid(axs,(1,6),colspan=2)
ax_10 = plt.subplot2grid(axs,(1,8),colspan=2)
        
ax_11 = plt.subplot2grid(axs,(2,1),colspan=4)
ax_12 = plt.subplot2grid(axs,(2,6),colspan=4)

ax_10.set_xticks([0,0.3,1])

types = ['Tc','r','GFf','ss','sc']
axes_0 = [ax_1,ax_2,ax_3,ax_4,ax_5]
axes_1 = [ax_6,ax_7,ax_8,ax_9,ax_10]
names = [r'$\tau$',r'$f$',r'$g$',r'$\sigma_s$',r'$\sigma_c$']


hist = dict()
mhist = dict()
scalef = []
for typ,ax_s in zip(types,axes_0):
        try:
            hist[typ+"o"] = np.histogram(data[typ].loc[idx['old',p['ss'],p['sc'],p['m'],p['GFf']]],density=True)
        except:
            pass
        hist[typ+"f"] = np.histogram(data[typ].loc[idx['full',p['ss'],p['sc'],p['m'],p['GFf']]],density=True)
    
for typ,ax_s in zip(types,axes_0):
        try:
            bins = np.linspace(hist[typ+'o'][1][0],hist[typ+'o'][1][-1],len(hist[typ+'o'][1])*2)
            hist[typ+"o"] = ax_s.hist(data[typ].loc[idx['old',p['ss'],p['sc'],p['m'],p['GFf']]],\
                                      bins=bins,alpha=alpha_no ,density=True,color=color_no,ec=color_no,linewidth=0.5)
            ax_s.hist(data[typ].loc[idx['old',p['ss'],p['sc'],p['m'],p['GFf']]],\
                      bins=bins,density=True,histtype='step',color=color_no,linewidth=1.0)
            mhist[typ+"o"] = hist[typ+"o"][0].max()
        except:
            mhist[typ+"o"] = 0
            pass
        bins = np.linspace(hist[typ+'f'][1][0],hist[typ+'f'][1][-1],len(hist[typ+'f'][1])*2)
        ax_s.hist(data[typ].loc[idx['full',p['ss'],p['sc'],p['m'],p['GFf']]],\
                  bins=bins,density=True,histtype='step',color=color_pr,linewidth=1.0)
        hist[typ+"f"] = ax_s.hist(data[typ].loc[idx['full',p['ss'],p['sc'],p['m'],p['GFf']]],\
                                  bins=bins,alpha=alpha_pr ,density=True,color=color_pr,edgecolor=color_pr,linewidth=0.5)

        mhist[typ+"f"] = hist[typ+"f"][0].max()
        
        scalef.append(mhist[typ+"f"] < mhist[typ+"o"])
        
scalef = np.max( scalef ) 
for typ,ax_s,name in zip(types,axes_0,[r'$\tau$',r'$f$',r'$g$',r'$\sigma_s$',r'$\sigma_c$']):
    if scalef:
        a = 'o'
        b = 'f'
    else:
        a = 'f'
        b = 'o'
    if mhist[typ+a] == 0:
        ax_s.set_ylim(0,mhist[typ+b]*1/0.9)
        ymax = mhist[typ+b]*1/0.9
    else:
        ax_s.set_ylim(0,mhist[typ+a]*1/0.9)
        ymax = mhist[typ+a]*1/0.9
    try:
        xo = np.mean(data[typ].loc[idx['old',p['ss'],p['sc'],p['m'],p['GFf']]])
        ind = np.sum(hist[typ+'o'][1]<xo)-1
        ax_s.plot([xo,xo],[0, ymax*0.249 ],"-",color=color_no,lw=1.2,zorder=1.1)      
        ax_s.plot([xo,xo],[0, ymax*0.25],"-",lw=2,color='.15',zorder=1)
    except:
        pass
    aco = sdata[typ+'_p'].loc[idx['old',p['ss'],p['sc'],p['m'],p['GFf']]]
    acn = sdata[typ+'_p'].loc[idx['full',p['ss'],p['sc'],p['m'],p['GFf']]]
    ax_s.set_title(r'{}'.format(name)  )
    xf = np.mean(data[typ].loc[idx['full',p['ss'],p['sc'],p['m'],p['GFf']]])
    ind = np.sum(hist[typ+'f'][1]<xf)-1
    ax_s.plot([xf,xf],[0, ymax*0.249 ],"-",color=color_pr,lw=1.2,zorder=1.1)      
    ax_s.plot([xf,xf],[0, ymax*0.25 ],"-",color='.15',lw=2,zorder=1)        
    #ax_s.plot([p[typ],p[typ]],[0,ymax*0.2],"-",color='gray',lw=4,zorder=8)
    ax_s.set_yticks(np.linspace(0,1,5)*ymax)
    ax_s.set_yticklabels([])
    
    

 

data,sdata = cla.utils.calc_statistic(data_s,p)



hist = dict()
mhist = dict()
scalef = []
for typ,ax_s in zip(types,axes_1):
        try:
            hist[typ+"o"] = np.histogram(data[typ].loc[idx['old',p['ss'],p['sc'],p['m'],p['GFf']]],density=True)
        except:
            pass
        hist[typ+"f"] = np.histogram(data[typ].loc[idx['full',p['ss'],p['sc'],p['m'],p['GFf']]],density=True)
    

    
for typ,ax_s in zip(types,axes_1):
        try:
            bins = np.linspace(hist[typ+'o'][1][0],hist[typ+'o'][1][-1],len(hist[typ+'o'][1])*2)
            hist[typ+"o"] = ax_s.hist(data[typ].loc[idx['old',p['ss'],p['sc'],p['m'],p['GFf']]],\
                                      bins=bins,alpha=alpha_no ,density=True,color=color_no,ec=color_no,linewidth=0.5)
            ax_s.hist(data[typ].loc[idx['old',p['ss'],p['sc'],p['m'],p['GFf']]],\
                      bins=bins,density=True,histtype='step',color=color_no,linewidth=1.0)
            mhist[typ+"o"] = hist[typ+"o"][0].max()
        except:
            mhist[typ+"o"] = 0
            pass
        bins = np.linspace(hist[typ+'f'][1][0],hist[typ+'f'][1][-1],len(hist[typ+'f'][1])*2)
        ax_s.hist(data[typ].loc[idx['full',p['ss'],p['sc'],p['m'],p['GFf']]],\
                  bins=bins,density=True,histtype='step',color=color_pr,linewidth=1.0)
        hist[typ+"f"] = ax_s.hist(data[typ].loc[idx['full',p['ss'],p['sc'],p['m'],p['GFf']]],\
                                  bins=bins,alpha=alpha_pr ,density=True,color=color_pr,ec=color_pr,lw=0.5)

        mhist[typ+"f"] = hist[typ+"f"][0].max()
        
        scalef.append([mhist[typ+"f"] < mhist[typ+"o"]])
        
scalef = np.max( scalef )
for typ,ax_s,name in zip(types,axes_1,names):
    if scalef:
        a = 'o'
        b = 'f'
    else:
        a = 'f'
        b = 'o'
    if mhist[typ+a] == 0:
        ax_s.set_ylim(0,mhist[typ+b]*1/0.9)
        ymax = mhist[typ+b]*1/0.9
    else:
        ax_s.set_ylim(0,mhist[typ+a]*1/0.9)
        ymax = mhist[typ+a]*1/0.9
    try:
        xo = np.mean(data[typ].loc[idx['old',p['ss'],p['sc'],p['m'],p['GFf']]])
        ind = np.sum(hist[typ+'o'][1]<xo)-1
        ax_s.plot([xo,xo],[0, ymax*0.249 ],"-",color=color_no,lw=1.2,zorder=1.1)      
        ax_s.plot([xo,xo],[0, ymax*0.25],"-",lw=2,color='.15',zorder=1)
    except:
        pass
    aco = sdata[typ+'_p'].loc[idx['old',p['ss'],p['sc'],p['m'],p['GFf']]]
    acn = sdata[typ+'_p'].loc[idx['full',p['ss'],p['sc'],p['m'],p['GFf']]]
    #ax_s.set_title(r'{}'.format(name)  )
    xf = np.mean(data[typ].loc[idx['full',p['ss'],p['sc'],p['m'],p['GFf']]])
    ind = np.sum(hist[typ+'f'][1]<xf)-1
    ax_s.plot([xf,xf],[0, ymax*0.249 ],"-",color=color_pr,lw=1.2,zorder=1.1)      
    ax_s.plot([xf,xf],[0, ymax*0.25 ],"-",color='.15',lw=2,zorder=1)      
    #ax_s.plot([p[typ],p[typ]],[0,ymax*0.2],"-",color='gray',lw=4,zorder=8)
    ax_s.set_yticks(np.linspace(0,1,5)*ymax)
    ax_s.set_yticklabels([])
    
    

    
            
with pd.HDFStore('paper_ab',complevel=9) as store:
     allimnew = store['s100_n10'].loc[idx[:,:0.2:,:],:].copy()

allimnew.reset_index(inplace=True)
allimnew.rename({"level_0":"model"}, axis="columns",inplace=True)
allimnew.set_index(['model','start','leng','N'],inplace=True)
allimnew.sort_index(inplace=True)

        
im = cla.plot.plot2d_pandas(allimnew.loc[idx['old']],'Tc','start','leng',aspect='auto',vmin=0.8,vmax=1.2,\
        cmap=plt.cm.seismic,axis=ax_12)
plt.colorbar(im,ax=ax_12,extend='max')

im = cla.plot.plot2d_pandas(allimnew.loc[idx['full']],'Tc','start','leng',aspect='auto',vmin=0.8,vmax=1.2,\
        cmap=plt.cm.seismic,axis=ax_11)
#plt.colorbar(im,ax=ax_11,extend='neither')

ax_11.set_ylabel('measurment duration')
ax_12.set_ylabel('measurment duration')
ax_11.set_xlabel('time of first measurment')
ax_12.set_xlabel('time of first measurment')

#add lines
for azoom,aall in zip(axes_0,axes_1):
    azoom.set_zorder(10)
    aall.set_zorder(6)
    cla.plot.zoom_effect02(azoom,aall,color='black',ls=':',zorder=0)

    
#add text
ax_1.set_ylabel('probability density')
bb = ax_1.get_position()
atxt = plt.figtext(bb.x0 - ho_abc/fb,bb.y1 + 0.05/fh,'A - 100 samples',size='x-large',zorder=7)

ax_6.set_ylabel('probability density')
bb = ax_6.get_position()
btxt = plt.figtext(bb.x0 - ho_abc/fb,bb.y1 + 0.05/fh,'B - 5 samples',size='x-large',zorder=7)
btxt.set_bbox(dict(facecolor='white', alpha=1, edgecolor=None,pad=0))



#ax_tmp = plt.subplot2grid(axs,(2,0))
#ax_tmp.set_axis_off()
#bb = ax_tmp.get_position()
#plt.figtext(bb.x0 - ho_abc/fb,bb.y1 + 0.05/fh,'C',size='x-large')

bb = ax_11.get_position()
plt.figtext(bb.x0 - 0.7/fb,bb.y1 + 0.05/fh,'C',size='x-large')
bb = ax_12.get_position()
plt.figtext(bb.x0 - 0.7/fb,bb.y1 + 0.05/fh,'D',size='x-large')

plt.draw()
        
tmp = ax_4.get_xticklabels()
tmp[-2].set_horizontalalignment('right')
ax_4.set_xticklabels(tmp)

tmp = ax_5.get_xticklabels()
tmp[1].set_horizontalalignment('left')
ax_5.set_xticklabels(tmp)

tmp = ax_8.get_xticklabels()
tmp[-2].set_horizontalalignment('right')
ax_8.set_xticklabels(tmp)

for ax_s in axes_0:
    for t in ax_s.get_xticklabels():
        #try:
        #    bbox = t.get_window_extent().transformed(ax_s.transData.inverted())
        #except:
        #    continue
        #rect = mpl.patches.Rectangle((bbox.x0,bbox.y0), bbox.x1 - bbox.x0 ,bbox.y1 - bbox.y0 , ec="none", color='white',zorder=0)
        #rect.set_clip_on(False)
        #ax_s.add_patch(rect)
        t.set_bbox(dict(facecolor='white', alpha=1, edgecolor=None,pad=0))


for typ,ax_s in zip(types,axes_0):
    labels = []
    trans = plttransforms.blended_transform_factory(ax_s.transData, ax_s.transAxes)
    for l,t in zip(ax_s.get_xticklabels(),ax_s.get_xticks()):
        la = l.get_text()
        if np.abs(t - p[typ])<0.005:
            lold = la.replace('$','')
            #lfont = l.get_font_properties()
            #lfont.set_weight('bold')
            #l.set_fontproperties(font)
            la = '$\\mathbf{'+lold+'}$'
            line = pltlines.Line2D([t,t],[0,-0.045],color='0.15',transform=trans,clip_on=False,lw=2,solid_capstyle='butt' )
            ax_s.add_line(line)
        labels.append(la)
    ax_s.set_xticklabels(labels)
        
for typ,ax_s in zip(types,axes_1):
    labels = []
    trans = plttransforms.blended_transform_factory(ax_s.transData, ax_s.transAxes)
    for l,t in zip(ax_s.get_xticklabels(),ax_s.get_xticks()):
        la = l.get_text()
        if np.abs(t - p[typ])<0.005:
            lold = la.replace('$','')
            #lfont = l.get_font_properties()
            #lfont.set_weight('bold')
            #l.set_fontproperties(font)
            la = '$\\mathbf{'+lold+'}$'
            line = pltlines.Line2D([t,t],[0,-0.045],color='0.15',transform=trans,clip_on=False,lw=2,solid_capstyle='butt' )
            ax_s.add_line(line)
        labels.append(la)
    ax_s.set_xticklabels(labels)

bb = ax_2.get_position()
lax = plt.axes([bb.x0 - 0.2/fb,bb.y1 + 0.2/fh ,0.4,0.014])
lax.set_xticks([])
lax.set_yticks([])
lax.set_frame_on(False)
lax.set_xlim(0,100)
a = 20
a3 = 60
s = 1.2
at= 4

lax.text(a+at,0,'probabilistic model',size='large')
lax.bar(a+s*np.array([0,1]),[2.8,5],s*1,linewidth=0.5,color='w',edgecolor=color_pr,alpha=alpha_pr)
lax.plot(a+s*np.array([-0.50,-0.5,0.5,0.5,1.5,1.5]),[0,2.8,2.8,5,5,0],color=color_pr)
lax.bar(a+s*np.array([0,1]),[2.8,5],s*1,linewidth=0.0,color=color_pr,edgecolor=None,alpha=alpha_pr)

lax.text(a3+a+at,0,'Nowakowski model',size='large')
lax.bar(a3+a+s*np.array([0,1]),[2.8,5],s*1,linewidth=0.5,color='w',edgecolor=color_no,alpha=alpha_no)
lax.plot(a3+a+s*np.array([-0.50,-0.5,0.5,0.5,1.5,1.5]),[0,2.8,2.8,5,5,0],color=color_no)
lax.bar(a3+a+s*np.array([0,1]),[2.8,5],s*1,linewidth=0.0,color=color_no,edgecolor=None,alpha=alpha_no)

bb = ax_12.get_position()
ax_11.set_position([ax_11.get_position().x0,bb.y0,bb.width,bb.height])
    
plt.draw()